# Time-Series Forecasting of Economic Indicators

This notebook provides an example of how to use time-series analysis to forecast a key economic indicator. We will use a simple ARIMA model to forecast monthly unemployment data.

## 1. Load and Prepare the Data

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from statsmodels.tsa.arima.model import ARIMA
from statsmodels.tsa.stattools import adfuller

# For demonstration purposes, we will use a well-known dataset on unemployment from statsmodels
from statsmodels.datasets import macrodata

df = macrodata.load_pandas().data
df['date'] = pd.to_datetime(df['year'].astype(str) + 'Q' + df['quarter'].astype(str))
df = df.set_index('date')

unemp_ts = df['unemp']
unemp_ts.plot(title='US Unemployment Rate')
plt.show()

## 2. Test for Stationarity

ARIMA models require the time series to be stationary (i.e., the mean and variance are constant over time). We can use the Augmented Dickey-Fuller test to check for stationarity.

In [ ]:
def check_stationarity(timeseries):
    result = adfuller(timeseries)
    print('ADF Statistic: %f' % result[0])
    print('p-value: %f' % result[1])
    if result[1] <= 0.05:
        print("Data is stationary")
    else:
        print("Data is not stationary. Differencing may be required.")

check_stationarity(unemp_ts)

## 3. Fit the ARIMA Model

Since the data is not stationary, we will use differencing (the 'd' component of ARIMA). A simple ARIMA(5,1,0) model is often a good starting point for economic data.

In [ ]:
# Fit ARIMA model
# The order (p,d,q) corresponds to: (AR order, Differencing order, MA order)
model = ARIMA(unemp_ts, order=(5, 1, 0))
model_fit = model.fit()

# Print summary
print(model_fit.summary())

## 4. Forecast Future Values

In [ ]:
# Forecast
forecast = model_fit.get_forecast(steps=12) # Forecast next 12 quarters
forecast_index = pd.date_range(start=unemp_ts.index[-1], periods=13, freq='Q')[1:]
forecast_series = pd.Series(forecast.predicted_mean.values, index=forecast_index)
confidence_intervals = forecast.conf_int()

# Plot the forecast
plt.figure(figsize=(12,6))
plt.plot(unemp_ts, label='Historical')
plt.plot(forecast_series, label='Forecast')
plt.fill_between(forecast_index, confidence_intervals.iloc[:, 0], confidence_intervals.iloc[:, 1], color='k', alpha=.15)
plt.title('Unemployment Rate Forecast')
plt.legend()
plt.show()